In [1]:
# import Packages:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os, sys

# Get the current working directory
cwd = os.getcwd()
mydir = os.path.abspath(os.path.join(cwd, ".."))
sys.path.append(mydir)
from DataGenerator import genTrainValidFolds

In [2]:
STDbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Classification-1/STD/batchData/"
EDGbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Classification-1/EDG/batchData/"
HOGp1batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Classification-1/HOGp1/batchData/"



In [20]:
# def toOneHotVector():
    

def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
def reshape_data(dataset, labels, featureSize, numLabels, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape(sample_size, featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape(len(dataset), featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [51]:
class BuildNeuralNet():
    
    def __init__(self):
        
        seed = 2316
        self.featureSize = 162
        self.numHidden1 = 1000
        self.numHidden2 = 1000
        self.numHidden3 = 1000
        self.numLabels = 2
        self.alpha = 0.03
        self.momentum = 0.9
        
        self.weights = {
            "inp_to_hid1_wght": tf.Variable(tf.random_normal([self.featureSize, self.numHidden1], seed=seed)),
            "hid1_to_hid2_wght" : tf.Variable(tf.random_normal([self.numHidden1, self.numHidden2], seed=seed)),
            "hid2_to_hid3_wght": tf.Variable(tf.random_normal([self.numHidden2, self.numHidden3], seed=seed)),
            "hid3_to_out_wght": tf.Variable(tf.random_normal([self.numHidden3, self.numLabels], seed=seed))
        }
        
        self.biases = {
            "hid1_bias": tf.Variable(tf.random_normal([self.numHidden1], seed=seed)),
            "hid2_bias": tf.Variable(tf.random_normal([self.numHidden2], seed=seed)),
            "hid3_bias": tf.Variable(tf.random_normal([self.numHidden3], seed=seed)),
            "out_bias": tf.Variable(tf.random_normal([self.numLabels], seed=seed))
        }
        
        
    def nNet(self):
        trainData = tf.placeholder(tf.float32, [None, self.featureSize])
        trainLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # 1st Hidden Layer State
        inp_to_hid1 = tf.matmul(trainData, self.weights['inp_to_hid1_wght']) + self.biases['hid1_bias']
        hid1State = tf.sigmoid(inp_to_hid1)
        
        # 2nd Hidden Layer State
        hid1_to_hid2 = tf.matmul(hid1State, self.weights['hid1_to_hid2_wght']) + self.biases['hid2_bias']
        hid2State = tf.sigmoid(hid1_to_hid2)
        
        # 3nd Hidden Layer State
        hid2_to_hid3 = tf.matmul(hid2State, self.weights['hid2_to_hid3_wght']) + self.biases['hid3_bias']
        hid3State = tf.sigmoid(hid2_to_hid3)
        
        # Output Layer State
        hid3_to_out = tf.matmul(hid3State, self.weights['hid3_to_out_wght']) + self.biases['out_bias']
        outState = tf.nn.softmax(hid3_to_out)
        
        # Loss Function and Optimization
        lossCE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hid3_to_out, labels=trainLabels))
#         optimizer = tf.train.AdamOptimizer(learning_rate=self.alpha).minimize(lossCE)
        optimizer = tf.train.MomentumOptimizer(self.alpha, 
                                            self.momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(lossCE)
        
        # Evaluate model
        correct_pred = tf.equal(tf.argmax(outState, 1), tf.argmax(trainLabels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        return dict(
            trainData = trainData, 
            trainLabels = trainLabels,
            optimizer = optimizer,
            lossCE = lossCE,
            accuracy = accuracy
        )
        

In [52]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


epochs = 30
def train(graphDict):
    print (graphDict)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(HOGp1batch_dir, oneHot=True)):
            print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
            print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
            print ('The Label Dictionary is given as: ', labelDict)

#             batchData, batchLabels = reshape_data(trainDataIN, trainLabelsIN, featureSize=162, numLabels=2)
            for epoch in range(epochs): 
                feed_dict = {graphDict['trainData']: trainDataIN,
                             graphDict['trainLabels']: trainLabelsIN
                        }

                _, loss, acc = sess.run([graphDict['optimizer'], graphDict['lossCE'], graphDict['accuracy']], feed_dict=feed_dict)
                print ("Iter " + str(epoch) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
            break
#             epoch += 1
#         print ("Optimization Finished!")
#     obj_SVM = Models()
#     batchEvalDict[foldNUM] = obj_SVM.classify(trainData, trainLabels, validData)

reset_graph()
graphDict = BuildNeuralNet().nNet()
train(graphDict)

{'trainLabels': <tf.Tensor 'Placeholder_1:0' shape=(?, 2) dtype=float32>, 'accuracy': <tf.Tensor 'Mean_1:0' shape=() dtype=float32>, 'lossCE': <tf.Tensor 'Mean:0' shape=() dtype=float32>, 'trainData': <tf.Tensor 'Placeholder:0' shape=(?, 162) dtype=float32>, 'optimizer': <tensorflow.python.framework.ops.Operation object at 0x11b3c3d68>}
Running i is : 0
Validation Data and Labels shape:  (1000, 162) (1000, 2)
Training Data and Labels shape:  (9000, 162) (9000, 2)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Iter 0, Minibatch Loss= 8.119655, Training Accuracy= 0.50700
Iter 1, Minibatch Loss= 19.884375, Training Accuracy= 0.49989
Iter 2, Minibatch Loss= 2.548061, Training Accuracy= 0.61111
Iter 3, Minibatch Loss= 4.615665, Training Accuracy= 0.59778
Iter 4, Minibatch Loss= 7.296151, Training Accuracy= 0.52400
Iter 5, Minibatch Loss= 7.536325, Training Accuracy= 0.56444
Iter 6, Minibatch Loss= 2.648139, Training Accuracy= 0.71922
Iter 7, Min